# Set-up

In [13]:
from typing import List
import math
import tqdm

# PCA - principal component analysis

## From scratch

In [12]:
Vector = List[float]

def subtract(v: Vector, w: Vector) -> Vector:
    """Subtracts corresponding elements"""
    assert len(v) == len(w), "vectors must be the same length"
    return [v_i - w_i for v_i, w_i in zip(v, w)]

assert subtract([5, 7, 9], [4, 5, 6]) == [1, 2, 3]

def vector_sum(vectors: List[Vector]) -> Vector:
    """Sums all corresponding elements"""
    # Check that vectors is not empty
    assert vectors, "no vectors provided!"

    # Check the vectors are all the same size
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), "different sizes!"

    # the i-th element of the result is the sum of every vector[i]
    return [sum(vector[i] for vector in vectors)
            for i in range(num_elements)]

assert vector_sum([[1, 2], [3, 4], [5, 6], [7, 8]]) == [16, 20]

def scalar_multiply(c: float, v: Vector) -> Vector:
    """Multiplies every element by c"""
    return [c * v_i for v_i in v]

assert scalar_multiply(2, [1, 2, 3]) == [2, 4, 6]

def vector_mean(vectors: List[Vector]) -> Vector:
    """Computes the element-wise average"""
    n = len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))

def dot(v: Vector, w: Vector) -> float:
    """Computes v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w), "vectors must be same length"

    return sum(v_i * w_i for v_i, w_i in zip(v, w))

assert dot([1, 2, 3], [4, 5, 6]) == 32  # 1 * 4 + 2 * 5 + 3 * 6

def sum_of_squares(v: Vector) -> float:
    """Returns v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

assert sum_of_squares([1, 2, 3]) == 14  # 1 * 1 + 2 * 2 + 3 * 3

def magnitude(v: Vector) -> float:
    """Returns the magnitude (or length) of v"""
    return math.sqrt(sum_of_squares(v))   # math.sqrt is square root function

def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector:
    """Moves `step_size` in the `gradient` direction from `v`"""
    assert len(v) == len(gradient)
    step = scalar_multiply(step_size, gradient)
    return add(v, step)

In [6]:
def de_mean(data: List[List[float]]) -> List[List[float]]:
    '''Recenters the data to have mean 0 in every dimension'''
    mean = vector_mean(data)
    return [subtract(vector, mean) for vector in data]

In [7]:
def direction(w: Vector) -> Vector:
    mag = manitude(w)
    return [w_i / mag for w_i in w]

In [8]:
def directional_variance(data: List[Vector], w: Vector) -> float:
    '''Returns the variance of x in the direction of w'''
    w_dir = direction(w)
    return sum(dot(v, w_dir) ** 2 for v in data)

In [9]:
def directional_variance_gradient(data: List[Vector], w: Vector) -> Vector:
    '''The gradient of directional variance with respect to w'''
    w_dir = direction(w)
    return [sum(2 * dot(v, w_dir) * v[i] for v in data)
            for i in range(len(w))]

In [15]:
def first_principal_component(data: List[Vector],
                              n: int = 100,
                              step_size: float = 0.1) -> Vector:
    guess = [1.0 for _ in data[0]]
    
    with tqdm.trange(n) as t:
        for _ in t:
            dv = directional_variance(data, guess)
            gradient = directional_variance_gradient(data, guess)
            guess = gradient_step(guess, gradient, step_size)
            t.set_description(f'dv:{dv:.3f}')
            
    return direction(guess)

In [16]:
def project(v: Vector, w: Vector) -> Vector:
    '''Returns the projection of v onto the direction w'''
    projection_length = dot(v, w)
    return scalar_multiply(projection_length, w)

In [18]:
def remove_projection_from_vector(v: Vector, w: Vector) -> Vector:
    '''Projects v onto w and subtracts the result from v'''
    return subtract(v, project(v, w))

def remove_projection(data: List[Vector], w: Vector) -> List[Vector]:
    return [remove_projection_from_vector(v, w) for v in data]

In [19]:
def pca(data: List[Vector], num_components: int) -> List[Vector]: 
    components: List[Vector] = []
    for _ in range(num_components):
        components = first_principal_component(data)
        components.append(component)
        data = remove_projection(data, component)
    return components

In [20]:
def transform_vector(v: Vector, components: List[Vector]) -> Vector:
    return [dot(v, w) for w in components]

def transform(data: List[Vector], components: List[Vector]) -> List[Vector]:
    return [transform_vector(v, components) for v in data]